# Import data

In [10]:
!pip install nbformat

  Using cached nbformat-5.10.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached fastjsonschema-2.21.2-py3-none-any.whl.metadata (2.3 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
Using cached nbformat-5.10.4-py3-none-any.whl (78 kB)
Using cached fastjsonschema-2.21.2-py3-none-any.whl (24 kB)
Using cached typing_extensions-4.15.0-py3-none-any.whl (44 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [nbformat]7/8 [nbformat]ng]


In [1]:
import pandas as pd

apiControls= pd.read_json('Schemes/apiControls.json')
cisco = pd.read_csv('Schemes/Cisco.csv')
medinaEucs = pd.read_csv('Schemes/MEDINA_EUCS_ControlsMapping_Clean.csv')
spanishEns = pd.read_csv('Schemes/SpanishENS.csv')
secNumCloud = pd.read_csv('Schemes/secnumcloud_controlsParsedAndTranslatedEnglish.csv')

# Analizing relations

In [2]:
import pandas as pd
import plotly.graph_objects as go
import igraph as ig

# Creazione del grafo con igraph
g = ig.Graph(directed=True)

# Aggiunta nodi (dataframe)
dataframes = ["apiControls", "cisco", "medinaEucs", "secNumCloud", "spanishEns"]
g.add_vertices(dataframes)
g.vs["label"] = dataframes

# Aggiunta relazioni (edge) con descrizioni
relations = [
    ("cisco", "medinaEucs", "EUCS Control Codes"),
    ("cisco", "spanishEns", "Spanish ENS High Control"),
    ("cisco", "secNumCloud", "SecNumCloud Control"),
    ("cisco", "medinaEucs", "EUCS Basic Control"),
    ("apiControls", "cisco", "Control Names/IDs"),
    ("apiControls", "medinaEucs", "code"),
    ("medinaEucs", "secNumCloud", "SecNumCloud FRANCE"),

]
for src, dst, label in relations:
    g.add_edge(g.vs.find(src).index, g.vs.find(dst).index, label=label)

# Calcola il layout del grafo
layout = g.layout("kk")
pos = {name: layout[i] for i, name in enumerate(g.vs["label"])}

# Crea liste per le posizioni dei nodi
edge_x = []
edge_y = []
for edge in g.es:
    start_node = g.vs[edge.source]["label"]
    end_node = g.vs[edge.target]["label"]
    x0, y0 = pos[start_node]
    x1, y1 = pos[end_node]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

# Traccia gli archi
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

# Traccia i nodi
node_x = [pos[name][0] for name in dataframes]
node_y = [pos[name][1] for name in dataframes]
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    hoverinfo='text',
    text=dataframes,
    textposition="bottom center",
    textfont=dict(size=14),
    marker=dict(
        showscale=False,
        size=25,
        color='#FFDDC1',
        line=dict(color='black', width=1.5)))

# Crea le etichette degli archi
edge_labels = []
for edge in g.es:
    start_node = g.vs[edge.source]["label"]
    end_node = g.vs[edge.target]["label"]
    label = edge["label"]
    x0, y0 = pos[start_node]
    x1, y1 = pos[end_node]
    edge_labels.append(go.Scatter(
        x=[(x0+x1)/2], y=[(y0+y1)/2],
        mode='text',
        text=f'<b>{label}</b>',  # Modifica qui per il grassetto
        textposition="middle center",
        showlegend=False,
        textfont=dict(size=12, color="gray")))

# Combina tutti i tracciati
fig = go.Figure(data=[edge_trace, node_trace] + edge_labels,
                layout=go.Layout(
                    title={
                        'text': 'Relazioni tra DataFrame (Plotly)',
                        'font': {'size': 16}
                    },
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20, l=5, r=5, t=40),
                    annotations=[dict(
                        ax=edge_x[i*3], ay=edge_y[i*3],
                        axref='x', ayref='y',
                        x=edge_x[i*3+1], y=edge_y[i*3+1],
                        xref='x', yref='y',
                        showarrow=True,
                        arrowhead=2,
                        arrowsize=1,
                        arrowwidth=1,
                        arrowcolor='#888') for i in range(len(relations))],
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

fig.show()